<h1><b>Restricted Boltzmann Machine και Deep Belief Network</b></h1>
<p align="justify">Στην συγκεκριμένη άσκηση θα μελετήσετε τον τρόπο λειτουργίας μιας <i>RBM (<a href="https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine">Restricted Boltzmann Machine</a>)</i> καθώς και των <i>DBN (<a href="https://en.wikipedia.org/wiki/Deep_belief_network">Deep Belief Network</a>)</i>, χρησιμοποιώντας το έτοιμο πρόγραμμα που σας δίνεται.Το συγκεκριμένο πρόγραμμα αξιοποιεί το <a href="https://en.wikipedia.org/wiki/MNIST_database">dataset του <i>MNIST</i></a>, όπου είναι μια μεγάλη βάση δεδομένων με χειρόγραφα ψηφία που χρησιμοποιείται συνήθως για την εκπαίδευση διαφόρων συστημάτων επεξεργασίας εικόνας. Για την άσκηση, θα πρέπει να χρησιμοποιήσετε το αρχείο <i>mnist_original.mat</i>, το οποίο είναι διαθέσιμο από <a href="https://www.kaggle.com/datasets/avnishnish/mnist-original?resource=download">εδώ</a>.</p>
<p align="justify">Μία αρκετά σημαντική εφαρμογή της <i>RBM</i> είναι η εξαγωγή χαρακτηριστικών (feature representation) από ένα dataset με σκοπό την αναπαράσταση της εισόδου (ορατοί νευρώνες) με ένα διάνυσμα μικρότερης διάστασης (κρυφοί νευρώνες). Στη συγκεκριμένη άσκηση θα συγκρίνετε την ακρίβεια ενός ταξινομητή ψηφίων με τη χρήση του αλγορίθμου <i>Logistic Regression</i>, όταν εκείνος δέχεται ως είσοδο το dataset (i) χωρίς να έχει υποστεί επεξεργασία από το <i>RBM</i>, (ii) αφου υποστεί επεξεργασία από το <i>RBM</i>, (iii) με τη χρήση <i>DBN</i>, δηλαδή δύο stacked <i>RBM</i>.</p>
<p align="justify"> Με βάση τον κώδικα που σας έχει δοθεί, καλείστε να απαντήσετε στα παρακάτω ερωτήματα:</p>
<ul>
<li>Να περιγράψετε σύντομα τον τρόπο λειτουργίας μιας <i>RBM</i>. Τι διαφορές έχει σε σχέση με μία <i> Μηχανή Boltzmann</i>;</li>
<li>Ποια είναι η λογική των <i>DBN</i> και σε τι προβλήματα τα αξιοποιούμε;</li>
<li>Να αναφέρετε τις βασικότερες εφαρμογές των <i>RBM</i> και <i>DBN</i>.</li>
<li>Εκτός από <i>RBM</i>, τι άλλα μοντέλα μπορούν να χρησιμοποιηθούν για να δημιουργήσουν <i>DBN</i>.</li>
<li>Συγκρίνετε τα αποτελέσματα της ταξινόμησης με τον αλγόριθμo <i>Logistic Regression</i> χωρίς τη χρήση <i>RBM</i> σε σχέση με τα αποτελέσματα της ταξινόμησης που έχει χρησιμοποιηθεί η <i>RBM</i> καθώς και με αυτή όπου χρησιμοποιούνται <i>RBM</i> και <i>DBN</i> για την εξαγωγή των χαρακτηριστικών. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;</li>
</ul>

In [3]:
#!/usr/bin/env python
# source: https://devdreamz.com/question/905929-stacking-rbms-to-create-deep-belief-network-in-sklearn

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

def norm(arr):
    arr = arr.astype(float)
    arr -= arr.min()
    arr /= arr.max()
    return arr

if __name__ == '__main__':

    # load MNIST data set
    mnist = loadmat("mnist-original.mat")
    X, Y = mnist["data"].T, mnist["label"][0]

    # normalize inputs to 0-1 range
    X = norm(X)

    # split into train, validation, and test data sets
    X_train, X_test, Y_train, Y_test = train_test_split(X,       Y,       test_size=10000, random_state=0)
    X_train, X_val,  Y_train, Y_val  = train_test_split(X_train, Y_train, test_size=10000, random_state=0)

    # --------------------------------------------------------------------------------
    # set hyperparameters

    learning_rate = 0.02 
    total_units   =  800 
    total_epochs  =   50
    batch_size    =  128

    C = 100. # optimum for benchmark model according to sklearn docs: https://scikit-learn.org/stable/auto_examples/neural_networks/plot_rbm_logistic_classification.html#sphx-glr-auto-examples-neural-networks-plot-rbm-logistic-classification-py)

    # --------------------------------------------------------------------------------
    # construct models

    # RBM
    rbm = BernoulliRBM(n_components=total_units, learning_rate=learning_rate, batch_size=batch_size, n_iter=total_epochs, verbose=1)

    # "output layer"
    logistic = LogisticRegression(C=C, solver='lbfgs', multi_class='multinomial', max_iter=200, verbose=1)

    models = []
    models.append(Pipeline(steps=[('logistic', clone(logistic))]))                                              # base model / benchmark
    models.append(Pipeline(steps=[('rbm1', clone(rbm)), ('logistic', clone(logistic))]))                        # single RBM
    models.append(Pipeline(steps=[('rbm1', clone(rbm)), ('rbm2', clone(rbm)), ('logistic', clone(logistic))]))  # RBM stack / DBN

    # --------------------------------------------------------------------------------
    # train and evaluate models

    for model in models:
        # train
        model.fit(X_train, Y_train)

        # evaluate using validation set
        print("Model performance:\n%s\n" % (
            classification_report(Y_        val, model.predict(X_val))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\User\Downloads\DSML MsC\2nd semester\Στοχαστικές Διεργασίες και Βελτιστοποίηση\StochasticsLabPublic\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96       995
         1.0       0.96      0.98      0.97      1121
         2.0       0.90      0.90      0.90      1015
         3.0       0.90      0.88      0.89      1033
         4.0       0.93      0.92      0.92       976
         5.0       0.90      0.88      0.89       884
         6.0       0.94      0.94      0.94       999
         7.0       0.92      0.93      0.92      1034
         8.0       0.88      0.87      0.87       923
         9.0       0.89      0.90      0.90      1020

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000


[BernoulliRBM] Iteration 1, pseudo-likelihood = -141.39, time = 17.17s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -116.86, time = 26.82s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -107.32, time = 23.58s
[Bernoul

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\User\Downloads\DSML MsC\2nd semester\Στοχαστικές Διεργασίες και Βελτιστοποίηση\StochasticsLabPublic\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98       995
         1.0       0.99      0.99      0.99      1121
         2.0       0.96      0.97      0.97      1015
         3.0       0.97      0.96      0.97      1033
         4.0       0.98      0.97      0.97       976
         5.0       0.97      0.97      0.97       884
         6.0       0.98      0.98      0.98       999
         7.0       0.97      0.98      0.98      1034
         8.0       0.97      0.96      0.97       923
         9.0       0.95      0.97      0.96      1020

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000


[BernoulliRBM] Iteration 1, pseudo-likelihood = -140.48, time = 16.21s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -119.74, time = 22.70s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -105.57, time = 22.65s
[Bernoul

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\User\Downloads\DSML MsC\2nd semester\Στοχαστικές Διεργασίες και Βελτιστοποίηση\StochasticsLabPublic\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       995
         1.0       0.99      0.99      0.99      1121
         2.0       0.97      0.97      0.97      1015
         3.0       0.98      0.97      0.97      1033
         4.0       0.98      0.97      0.97       976
         5.0       0.97      0.97      0.97       884
         6.0       0.99      0.98      0.98       999
         7.0       0.97      0.98      0.97      1034
         8.0       0.97      0.96      0.96       923
         9.0       0.96      0.96      0.96      1020

    accuracy                           0.98     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.98      0.98      0.98     10000


